**Notebook preparation and importing libraries**

In [1]:
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid
from hvac.logging import ModuleLogger

In [2]:
logger = ModuleLogger.get_logger('hvac.fluids.fluid')
logger.setLevel(ModuleLogger.ERROR)

In [3]:
Q_ = Quantity

In [4]:
import hvac.heat_transfer.heat_exchanger.fin_tube.air_condenser.rating as rating

# Rating of a plain fin-tube counterflow air condenser

Define the type of refrigerant:

In [5]:
R134a = Fluid('R134a')

Define the type of condenser: a plain fin-tube, counterflow heat exchanger.

In [6]:
Condenser = rating.PFT_CO_CND

Create an instance of the condenser model, specifying the dimensions of the frontal area, the number of rows, the geometrical characteristics of the heat transfer surfaces, and the thermal conductivity of the fins (by default aluminium fins are assumed):

In [7]:
condenser = Condenser(
    L1=Q_(0.550, 'm'),
    L3=Q_(0.275, 'm'),
    N_r=5,
    S_t=Q_(25.4, 'mm'),
    S_l=Q_(22.0, 'mm'),
    D_i=Q_(8.422, 'mm'),
    D_o=Q_(10.2, 'mm'),
    t_f=Q_(0.3302, 'mm'),
    N_f=1 / Q_(3.175, 'mm')
)

On this instance, we set the known operating conditions:
- the mass flow rate of air through the condenser
- the mass flow rate of refrigerant, which was determined when rating the evaporator
- the state of air entering the condenser
- the state of refrigerant entering the condenser

The result of rating the condenser will contain:
- the state of refrigerant leaving the condenser
- the state of air leaving the condenser
- the heat rejection rate (the heat transfer rate from the refrigerant to the air stream)
- the degree of subcooling
- the flow length of air through the desuperheating, condensing, and subcooling part of the condenser

The rating routine (of the subcooling part) is an iterative routine. The criterion to stop the iterative routine is the tolerable deviation between the last and the previous calculated heat exchanger effectiveness, which can be set through parameter `tol_eps`. Should the tolerable deviation not be reached after a maximum number of iterations, which can be set with parameter `i_max`, a `ValueError` exception will be raised and no result is returned.

In [8]:
r = condenser(
    m_dot_air=Q_(2454.820, 'kg / hr'),
    m_dot_rfg=Q_(133.227, 'kg / hr'),
    air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    rfg_in=R134a(T=Q_(74.32, 'degC'), P=Q_(14.92, 'bar'))
)

In [9]:
print(
    f"air out = {r.air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{r.air_out.RH.to('pct'):~P.0f} RH\n"
    f"refrigerant out = {r.rfg_out.T.to('degC'):~P.1f}\n"
    f"degree of subcooling = {r.dT_subcooling.to('K'):~P.2f}\n"
    f"heat rejection rate = {r.Q.to('kW'):~P.3f}\n"
    f"L2 desuperheating = {r.L2_desuperheating.to('mm'):~P.0f}\n"
    f"L2 condensing = {r.L2_condensing.to('mm'):~P.0f}\n"
    f"L2 subcooling = {r.L2_subcooling.to('mm'):~P.0f}"
)

air out = 44.5 °C DB, 18 % RH
refrigerant out = 49.5 °C
degree of subcooling = 5.49 K
heat rejection rate = 6.582 kW
L2 desuperheating = 25 mm
L2 condensing = 73 mm
L2 subcooling = 12 mm
